In [2]:
# Import ecery library what you need for a fully connected neural network
# hint: 3 package + 1 for drawing nice things into the universe
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

# As you know from the before classes you need a dataset, so please import
# the MNIST dataset
#hint: from, import
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

# Read the dataset into a variable and (pls) use the one hot label encoding
# hint: one function calling with two parameters where the first is the
# 'MNIST_data/' folder(?) and the sec param is one_hot=...
data = mnist_data.read_data_sets('MNIST_data/', one_hot=True)

# Print out the images' shape and the labels' shape
# hint: you can find shape property in the data.train.whatyouneed.shape
# print(data)
print(data.train.images.shape, data.train.labels.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10)


In [16]:
# I would like to ask you to plot one picture to me.
# hint: reshape an image, because maybe it is too big or something.
# Ok, we need just smaller picture, it can be 28x28.
#img = np.reshape(data.train.images[0, :], (28, 28))
#plt.figure()
#plt.imshow(img)
#plt.show()

# Define two integer what are the input_size and output_classes
# hint: check again the shapes
input_size = data.train.images.shape[1]
output_size = data.train.labels.shape[1]

# Call the reset default graph method. You will be grateful later
# hint: tensorflow's function
tf.reset_default_graph()

# Save out into two variables the input images and the labels
# hint: 2 placeholders, float32, predefine the shapes
inputs = tf.placeholder(tf.float32, name="input",
                       shape=[None, input_size])
labels = tf.placeholder(tf.float32, name="output",
                        shape=[None, output_size])

# Write two hidden layer. First contains 1024 and the second
# contains 2048 neurons. Use relu aas activation function.
# hint: variable_scope, weight, bias, matmul, add, nn.relu
first_layer_neurons = 1024
second_layer_neurons = 2048
with tf.variable_scope('fully_1'):
    w = tf.get_variable('weight', [input_size, first_layer_neurons])
    b = tf.get_variable('bias', [first_layer_neurons])
    
    mult = tf.matmul(inputs, w)
    biased = tf.add(mult, b)
    
    next_inp = tf.nn.relu(biased)

with tf.variable_scope('fully_2'):
    w = tf.get_variable('weight', [first_layer_neurons, second_layer_neurons])
    b = tf.get_variable('bias', [second_layer_neurons])
    
    mult = tf.matmul(next_inp, w)
    biased = tf.add(mult, b)
    
    next_inp = tf.nn.relu(biased) 
    
# Write the output layer for me *-*. I know you know but I must say it
# again... You don't have to use the activation function in this layer, 
# because it is your result.
with tf.variable_scope('output_layer'):
    w = tf.get_variable('weight', [second_layer_neurons, output_size])
    b = tf.get_variable('bias', [output_size])
    
    mult = tf.matmul(next_inp, w)
    biased = tf.add(mult, b)
    
final_output = biased
    
# Calculate loss with the help of softmax_cross_entropy that contains 
# the execution of softmax function on the output and the cross entropy 
# calculation between output and one_hot_labels
# hint: reduce_mean, losses.softmax...
losses = tf.reduce_mean(tf.losses.softmax_cross_entropy(labels, final_output),
                        name= 'losses')

# Definition of optimizer as a GradientDescentOptimizer with 1e-4 learning
# rate for the minimization of loss
training_steps = tf.train.GradientDescentOptimizer(1e-4).minimize(losses)

# Calculaltion of correct predictions
# hint: equal, argmax
correct_prediction = tf.equal(tf.argmax(final_output), tf.argmax(labels),
                              name='cor_pred')

# Calculation of accuracy
# hint: reduce_mean, cast
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),
                         name='acc')

# Write the run section where the nn will learn and it will get back
# the accuracy and the loss. Print out the step, loss accuracy together
# in every 100th steps. After the loop print out the acc and loss onto
# a figure.
init = tf.global_variables_initializer()
with tf.Session() as ss:
    ss.run(init)
    
    batch_length = 64
    num_iteration = 15000
    
    loss_plot = np.zeros(num_iteration)
    acc_plot = np.zeros(num_iteration)
    
    for index in range(num_iteration):
        used_in_batch = random.sample(range(data.train.images.shape[0]),
                                      batch_length)
        batch_in = data.train.images[used_in_batch, :]
        batch_out = data.train.labels[used_in_batch, :]
        
        _, acc, loss = ss.run([training_steps, accuracy, losses], feed_dict={
            inputs: batch_in,
            labels: batch_out
        })
        
        acc_plot[index] = acc
        loss_plot[index] = loss
        
        if index % 100 == 0:
            print('Step: ' + str(index) + 'Loss: ' + str(loss) 
                  + 'Acc: ' + str(acc))
            
plt.plot(acc_plot, 'b', loss_plot, 'r')

ValueError: Tensor("fully_1/weight:0", shape=(784, 1024), dtype=float32_ref) must be from the same graph as Tensor("Mean:0", shape=(), dtype=float32).